In [9]:
import torch
import time
from snn_delays.snn import SNN
from snn_delays.utils.dataset_loader import DatasetLoader
from snn_delays.utils.train_utils import train, get_device
from snn_delays.utils.test_behavior import tb_save_max_last_acc

'''
SHD dataset as in ablation study
'''

device = get_device()

# for reproducibility
torch.manual_seed(10)

dataset = 'shd'
total_time = 50
batch_size = 1024

# DATASET
DL = DatasetLoader(dataset=dataset,
                  caching='memory',
                  num_workers=0,
                  batch_size=batch_size,
                  total_time=total_time,
                  crop_to=1e6)
train_loader, test_loader, dataset_dict = DL.get_dataloaders()
          
num_epochs = 50

lr = 1e-3
# SNN CON DELAYS
taimu1 = time.time()

tau_m = 'normal'
ckpt_dir = 'exp3_shd50_rnn' 

snn = SNN(dataset_dict=dataset_dict, structure=(64, 2), connection_type='mf',
    delay=None, delay_type='', tau_m = tau_m,
    win=total_time, loss_fn='mem_sum', batch_size=batch_size, device=device,
    debug=False)
snn.multi_proj = 3
snn.set_network()
snn.to(device)

snn2 = SNN(dataset_dict=dataset_dict, structure=(64, 2), connection_type='f',
    delay=(48, 16), delay_type='h', tau_m = tau_m,
    win=total_time, loss_fn='mem_sum', batch_size=batch_size, device=device,
    debug=False)
snn2.set_network()
snn2.to(device)

snn3 = SNN(dataset_dict=dataset_dict, structure=(64, 2), connection_type='r',
    delay=None, delay_type='', tau_m = tau_m,
    win=total_time, loss_fn='mem_sum', batch_size=batch_size, device=device,
    debug=False)
snn3.set_network()
snn3.to(device)


Running on: cuda:0
[CropTime(min=0, max=1000000.0), ToFrame(sensor_size=(700, 1, 1), time_window=None, event_count=None, n_time_bins=50, n_event_bins=None, overlap=0, include_incomplete=False)]
<class 'list'>

[INFO] Delays: tensor([0])

[INFO] Delays i: tensor([0])

[INFO] Delays h: tensor([0])

[INFO] Delays o: tensor([0])
1000.0
Delta t: 20.0 ms
mean of normal: -0.541324854612918

[INFO] Delays: tensor([ 0, 16, 32])

[INFO] Delays i: tensor([0])

[INFO] Delays h: tensor([ 0, 16, 32])

[INFO] Delays o: tensor([0])
1000.0
Delta t: 20.0 ms
mean of normal: -0.541324854612918

[INFO] Delays: tensor([0])

[INFO] Delays i: tensor([0])

[INFO] Delays h: tensor([0])

[INFO] Delays o: tensor([0])
1000.0
Delta t: 20.0 ms
mean of normal: -0.541324854612918


SNN(
  (criterion): CrossEntropyLoss()
  (f0_f1): Linear(in_features=700, out_features=64, bias=False)
  (f1_f1): Linear(in_features=64, out_features=64, bias=False)
  (f1_f2): Linear(in_features=64, out_features=64, bias=False)
  (f2_f2): Linear(in_features=64, out_features=64, bias=False)
  (f2_o): Linear(in_features=64, out_features=20, bias=False)
)

In [10]:
a, b = snn.test(test_loader, only_one_batch=True)
a, b = snn2.test(test_loader, only_one_batch=True)
a, b = snn3.test(test_loader, only_one_batch=True)

Test Loss: 4.206374645233154
Avg spk_count per neuron for all 50 time-steps 3.169830322265625
Avg spk per neuron per layer [7.56695556640625, 5.11236572265625]
Test Accuracy of the model on the test samples: 4.199

Test Loss: 4.626893043518066
Avg spk_count per neuron for all 50 time-steps 3.119739532470703
Avg spk per neuron per layer [6.776031494140625, 5.7029266357421875]
Test Accuracy of the model on the test samples: 4.395

Test Loss: 3.1440536975860596
Avg spk_count per neuron for all 50 time-steps 3.0626449584960938
Avg spk per neuron per layer [7.2813873291015625, 4.9691925048828125]
Test Accuracy of the model on the test samples: 4.199



In [4]:
snn.h_sum_spikes_per_layer

tensor([495908., 335044.])

In [6]:
snn2.h_sum_spikes_per_layer

tensor([444074., 373747.])

In [7]:
snn3.h_sum_spikes_per_layer

tensor([477193., 325661.])

In [12]:
print(snn.test_spk_count)
print(snn2.test_spk_count)
print(snn3.test_spk_count)

[[0, 3.169830322265625]]
[[0, 3.119739532470703]]
[[0, 3.0626449584960938]]


In [2]:
train(snn, train_loader, test_loader, lr, num_epochs, dropout=0.0, 
    test_behavior=tb_save_max_last_acc, ckpt_dir=ckpt_dir, scheduler=(100, 0.95), test_every=1)

training shd50_l2_1d1.t7 for 50 epochs...
Epoch [1/50], learning_rates 0.001000, 0.100000


c:\Users\Alberto\anaconda3\envs\deepsnn\Lib\site-packages\torch\optim\lr_scheduler.py:143: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
c:\Users\Alberto\anaconda3\envs\deepsnn\Lib\site-packages\torch\optim\lr_scheduler.py:156: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to re

Step [2/7], Loss: 4.13771
l1_score: 0
Step [4/7], Loss: 4.11290
l1_score: 0
Step [6/7], Loss: 3.24335
l1_score: 0
Step [8/7], Loss: 3.22379
l1_score: 0
Time elasped: 48.00258803367615
Test Loss: 3.13360333442688
Avg spk_count per neuron for all 50 time-steps 4.5544633865356445
Avg spk per neuron per layer [10.229599160777385, 7.988253643992933]
Test Accuracy of the model on the test samples: 6.228

Model saved in  C:\Users\Alberto\OneDrive - UNIVERSIDAD DE SEVILLA\PythonData\Checkpoints\exp3_shd50_rnn
saving max acc: 6.22791519434629
Model saved in  C:\Users\Alberto\OneDrive - UNIVERSIDAD DE SEVILLA\PythonData\Checkpoints\exp3_shd50_rnn
Epoch [2/50], learning_rates 0.001000, 0.100000
Step [2/7], Loss: 3.13398
l1_score: 0
Step [4/7], Loss: 3.13944
l1_score: 0
Step [6/7], Loss: 3.12411
l1_score: 0
Step [8/7], Loss: 3.03960
l1_score: 0
Time elasped: 2.104907512664795
Test Loss: 3.035099188486735
Avg spk_count per neuron for all 50 time-steps 4.870103359222412
Avg spk per neuron per layer 

In [3]:
snn.test_spk_count

[[1, 4.554463201192579],
 [2, 4.870103384496466],
 [3, 5.01134261815371],
 [4, 5.03505445284894],
 [5, 5.052944870803887],
 [6, 5.13726741939046],
 [7, 5.253619837676679],
 [8, 5.2898768772084805],
 [9, 5.370943642336573],
 [10, 5.313612867159894],
 [11, 5.331903572217315],
 [12, 5.237618705830389],
 [13, 5.345037130079505],
 [14, 5.323257715878976],
 [15, 5.430180957376325],
 [16, 5.392641963891343],
 [17, 5.413122170384276],
 [18, 5.491431785556537],
 [19, 5.438167720295937],
 [20, 5.434606545384276],
 [21, 5.5489540774072434],
 [22, 5.604707859430212],
 [23, 5.549890956272085],
 [24, 5.582283154814488],
 [25, 5.666353798586573],
 [26, 5.615488005189929],
 [27, 5.596053376214664],
 [28, 5.677851700530035],
 [29, 5.7274407160998235],
 [30, 5.8257042982553005],
 [31, 5.775031746908128],
 [32, 5.815210564818905],
 [33, 5.779817938383392],
 [34, 5.7978860700088335],
 [35, 5.820830112632509],
 [36, 5.839053528047703],
 [37, 5.760730109871908],
 [38, 5.841595006073321],
 [39, 5.77890521477

In [4]:
snn.num_neurons_list

[64, 64]

In [10]:
x = torch.rand(1024, 64)
print(x.repeat(1, 3).shape)
print(x.repeat(1, 3).view(1024, -1).shape)

torch.Size([1024, 192])
torch.Size([1024, 192])


In [14]:
x = torch.rand(10, 4)
x

tensor([[0.6131, 0.1452, 0.7860, 0.9258],
        [0.0528, 0.9084, 0.4363, 0.9647],
        [0.1770, 0.9467, 0.0742, 0.7290],
        [0.0121, 0.6393, 0.0876, 0.3102],
        [0.8581, 0.3634, 0.9454, 0.9337],
        [0.5624, 0.9277, 0.4248, 0.9033],
        [0.9952, 0.3456, 0.8911, 0.0317],
        [0.9378, 0.7023, 0.0194, 0.6718],
        [0.4343, 0.8132, 0.6807, 0.2210],
        [0.7399, 0.4589, 0.8210, 0.1687]])

In [15]:
x.repeat(1, 3)

tensor([[0.6131, 0.1452, 0.7860, 0.9258, 0.6131, 0.1452, 0.7860, 0.9258, 0.6131,
         0.1452, 0.7860, 0.9258],
        [0.0528, 0.9084, 0.4363, 0.9647, 0.0528, 0.9084, 0.4363, 0.9647, 0.0528,
         0.9084, 0.4363, 0.9647],
        [0.1770, 0.9467, 0.0742, 0.7290, 0.1770, 0.9467, 0.0742, 0.7290, 0.1770,
         0.9467, 0.0742, 0.7290],
        [0.0121, 0.6393, 0.0876, 0.3102, 0.0121, 0.6393, 0.0876, 0.3102, 0.0121,
         0.6393, 0.0876, 0.3102],
        [0.8581, 0.3634, 0.9454, 0.9337, 0.8581, 0.3634, 0.9454, 0.9337, 0.8581,
         0.3634, 0.9454, 0.9337],
        [0.5624, 0.9277, 0.4248, 0.9033, 0.5624, 0.9277, 0.4248, 0.9033, 0.5624,
         0.9277, 0.4248, 0.9033],
        [0.9952, 0.3456, 0.8911, 0.0317, 0.9952, 0.3456, 0.8911, 0.0317, 0.9952,
         0.3456, 0.8911, 0.0317],
        [0.9378, 0.7023, 0.0194, 0.6718, 0.9378, 0.7023, 0.0194, 0.6718, 0.9378,
         0.7023, 0.0194, 0.6718],
        [0.4343, 0.8132, 0.6807, 0.2210, 0.4343, 0.8132, 0.6807, 0.2210, 0.4343,